# Code to make animations of PDES for Lecture 15-16, March 11 & 13, 2019


In [ ]:
# Making matrices for the discrete Laplacian operator 

import numpy as np
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg

# One dimension 
def make_A_1D(k):
    """Create the matrix of the discrete Laplacian operator in one dimension on a k-point grid.
    Parameters: 
      k: number of grid points.
    Outputs:
      A: the sparse k-by-k matrix representing the finite difference approximation to Laplace's equation.
    """
    # First make a list with one triple (row, column, value) for each nonzero element of A
    triples = []
    for x in range(k):

        # what row of the matrix is grid point x?
        row = x
        
        # the diagonal element in this row
        col = row
        triples.append((row, col, 2.0))
        # connect to grid neighbors in x dimension
        if x > 0:
            col = row - 1
            triples.append((row, col, -1.0))
        if x < k - 1:
            col = row + 1
            triples.append((row, col, -1.0))

    # Finally convert the list of triples to a scipy sparse matrix
    ndim = k
    rownum = [t[0] for t in triples]
    colnum = [t[1] for t in triples]
    values = [t[2] for t in triples]
    A = sparse.csr_matrix((values, (rownum, colnum)), shape = (ndim, ndim))
    
    return A 

In [ ]:
# Diffusion in 1D, with forward Euler, Dirichlet boundary conditions

import numpy as np
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
from scipy import integrate

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams['animation.ffmpeg_path'] = '/Users/gilbert/anaconda3/bin/ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

plt.rcParams['animation.ffmpeg_path'] = '/Users/gilbert/anaconda3/bin/ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)


# Diffusion in 1D, with forward Euler by hand
#   du / dt = c(x) * d^2u / dx^2,
#   with boundary values at endpoints of unit interval,
#   initial value 0 at all points in unit interval, 
#   and with boundary values at endpoints
#   and with varying thermal conductivity c(x)

# Parameters of discretization and the equation

xspan  = (0, 1)         # Left and right endpoints of the 1-D interval
k      = 10             # Number of discrete points on the line, not including fixed-temp endpoints
dx     = (xspan[1] - xspan[0]) / (k+1)   # Difference between adjacent discrete points on line

tspan  = (0, 60)        # Starting and stopping times
dt     = .1             # Length of time step for forward Euler ODE solution
tsteps = int((tspan[1] - tspan[0]) / dt) + 1   # Number of time steps

uleft  = 0              # Fixed temperature at left endpoint of interval
uright = 100            # Fixed temperature at right endpoint of interval
u0    = np.zeros(k)     # Initial value of interior points at time 0 

c = np.ones(k) * .005   # Thermal conductivity at each interior point

# Uncomment to make the left half much less conductive than the right half
midpoint = int(k/2)
# c[:midpoint] = .0005

# Get the matrix and the boundary-condition adjustment
A      = make_A_1D(k)
b      = np.zeros(k)
b[0]   = uleft
b[-1]  = uright

# For plotting, we include the endpoints in an array xplot of k+2 discrete points,
# and an array uplot of the k interior values along with the 2 boundary values.
xplot         = np.linspace(xspan[0], xspan[1], k+2)
uplot         = np.empty(k+2)
uplot[0]      = uleft   
uplot[1:k+1]  = u0
uplot[k+1]    = uright  

# First plot
fig = plt.figure()
plt.plot(xplot, uplot, 'r')
plt.axis([xspan[0], xspan[1], uleft, uright])
plt.xlabel('distance')
plt.ylabel('temperature')
plt.title('time %f' % (tspan[0]))
ax = plt.gca()
points = ax.lines[0]

def one_frame(num):
    global udot, u, uplot, points
    points.set_ydata(uplot)
    plt.gca().set_title('time %.1f' % (num * dt))
    udot = -c * (A @ u - b) / dx**2
    u    = u + udot * dt
    uplot[1:k+1] = u
    return points,

u = u0

diff_ani = animation.FuncAnimation(fig, one_frame, tsteps,
                                   interval=50, blit=True)
diff_ani.save('test.mp4', writer=writer)
        


In [ ]:
# Diffusion in 1D, with forward Euler, Neumann boundary condition at left endpoint

import numpy as np
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
from scipy import integrate

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams['animation.ffmpeg_path'] = '/Users/gilbert/anaconda3/bin/ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

plt.rcParams['animation.ffmpeg_path'] = '/Users/gilbert/anaconda3/bin/ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)


# Diffusion in 1D, with forward Euler by hand
#   du / dt = c(x) * d^2u / dx^2,
#   with boundary values at endpoints of unit interval,
#   initial value 0 at all points in unit interval, 
#   and with boundary values at endpoints
#   and with varying thermal conductivity c(x)

# Parameters of discretization and the equation

xspan  = (0, 1)         # Left and right endpoints of the 1-D interval
k      = 10             # Number of discrete points on the line, not including fixed-temp endpoints
dx     = (xspan[1] - xspan[0]) / (k+1)   # Difference between adjacent discrete points on line

tspan  = (0, 300)        # Starting and stopping times
dt     = .1             # Length of time step for forward Euler ODE solution
tsteps = int((tspan[1] - tspan[0]) / dt) + 1   # Number of time steps

uleft  = 0              # Fixed temperature at left endpoint of interval
uright = 100            # Fixed temperature at right endpoint of interval
u0    = np.zeros(k)     # Initial value of interior points at time 0 

c = np.ones(k) * .005   # Thermal conductivity at each interior point

# Uncomment to make the left half much less conductive than the right half
midpoint = int(k/2)
c[:midpoint] = .0005

# Get the matrix and the boundary-condition adjustment
A      = make_A_1D(k)
A[0,0] = 1              # Neumann boundary condition on left
b      = np.zeros(k)
# b[0]   = uleft        # Neumann boundary condition on left
b[-1]  = uright

# For plotting, we include the endpoints in an array xplot of k+2 discrete points,
# and an array uplot of the k interior values along with the 2 boundary values.
xplot         = np.linspace(xspan[0], xspan[1], k+2)
uplot         = np.empty(k+2)
uplot[1:k+1]  = u0
uplot[0]      = u0[0]   # Neumann boundary condition on left
uplot[k+1]    = uright  

# First plot
fig = plt.figure()
plt.plot(xplot, uplot, 'r')
plt.axis([xspan[0], xspan[1], uleft, uright])
plt.xlabel('distance')
plt.ylabel('temperature')
plt.title('time %f' % (tspan[0]))
ax = plt.gca()
points = ax.lines[0]

def one_frame(num):
    global udot, u, uplot, points
    points.set_ydata(uplot)
    plt.gca().set_title('time %.1f' % (num * dt))
    udot = -c * (A @ u - b) / dx**2
    u    = u + udot * dt
    uplot[1:k+1] = u
    uplot[0]     = u[0]     # Neumann boundary condition on left
    return points,

u = u0

diff_ani = animation.FuncAnimation(fig, one_frame, tsteps,
                                   interval=50, blit=True)
diff_ani.save('test.mp4', writer=writer)
        


In [ ]:
# Wave equation in 1D, fully discrete with centered differences in both time and space
# Sine wave as initial condition

import numpy as np
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
from scipy import integrate
from scipy.stats import norm

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams['animation.ffmpeg_path'] = '/Users/gilbert/anaconda3/bin/ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

# Wave equation in 1D, fully discrete with centered differences in both time and space
#   d^2u / dt^2 = c * d^2u / dx^2,
#   with boundary values of 0 at the endpoints of unit interval,
#   initial values for u to be specified in the interior of the unit interval,
#   initial values of 0 for du/dt at all points in unit interval, 
#   and (for now) constant propagation speed c.

# Parameters of discretization and the equation
xspan  = (0, 1)         # Left and right endpoints of the 1-D interval
dx     = .01           # Distance between discrete points on the line
k      = int((xspan[1] - xspan[0]) / dx) - 1   # Number of discrete points in space, not including endpoints

tspan  = (0, 2)         # Starting and stopping times
dt     = .002           # Length of time step 
tsteps = int((tspan[1] - tspan[0]) / dt) + 1   # Number of time steps

c = 1                   # Speed of wave propagation

# Get the matrix for the discrete Laplace operator
A      = make_A_1D(k)

# Initial condition: value of interior points at time 0
u0 = np.sin(np.linspace(0,1,k)*np.pi*4)

# Intialize u and uprev to the same value, 
# because of the initial condition du/dt = 0 everywhere
u     = u0
uprev = u

# For plotting, we include the endpoints in an array xplot of k+2 discrete points,
# and an array uplot of the k interior values along with the 2 boundary values of 0.
xplot         = np.linspace(xspan[0], xspan[1], k+2)
uplot         = np.zeros(k+2)
uplot[1:k+1]  = u

# First plot
fig = plt.figure()
plt.plot(xplot, uplot, 'r')
plt.axis([xspan[0], xspan[1], -2, 2])
plt.xlabel('x')
plt.ylabel('u')
plt.title('time %f' % (tspan[0]))
ax = plt.gca()
points = ax.lines[0]

def one_frame(num):
    global u, uprev, uplot, points
    points.set_ydata(uplot)
    plt.gca().set_title('time %.1f' % (num * dt))
    unew  = 2*u  -  uprev  -  c * (dt/dx)**2 * A @ u
    uprev = u
    u     = unew
    uplot[1:k+1] = u
    return points,

u = u0

wave_ani = animation.FuncAnimation(fig, one_frame, tsteps,
                                   interval=50, blit=True)
wave_ani.save('test.mp4', writer=writer)
        

In [ ]:
# Wave equation in 1D, fully discrete with centered differences in both time and space
# Initial condition is one Gaussian wave in the middle

import numpy as np
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
from scipy import integrate
from scipy.stats import norm

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams['animation.ffmpeg_path'] = '/Users/gilbert/anaconda3/bin/ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

# Wave equation in 1D, fully discrete with centered differences in both time and space
#   d^2u / dt^2 = c * d^2u / dx^2,
#   with boundary values of 0 at the endpoints of unit interval,
#   initial values for u to be specified in the interior of the unit interval,
#   initial values of 0 for du/dt at all points in unit interval, 
#   and (for now) constant propagation speed c.

# Parameters of discretization and the equation
xspan  = (0, 1)         # Left and right endpoints of the 1-D interval
dx     = .001           # Distance between discrete points on the line
k      = int((xspan[1] - xspan[0]) / dx) - 1   # Number of discrete points in space, not including endpoints

tspan  = (0, 4)          # Starting and stopping times
dt     = .0005           # Length of time step 
tsteps = int((tspan[1] - tspan[0]) / dt) + 1   # Number of time steps

c = 1                   # Speed of wave propagation

# Get the matrix for the discrete Laplace operator
A      = make_A_1D(k)

# Initial condition: value of interior points at time 0
midpoint = int(k/2)
u0 =   100 * norm.pdf(range(k), loc = midpoint, scale = k/50)

# Intialize u and uprev to the same value, 
# because of the initial condition du/dt = 0 everywhere
u     = u0
uprev = u

# For plotting, we include the endpoints in an array xplot of k+2 discrete points,
# and an array uplot of the k interior values along with the 2 boundary values of 0.
xplot         = np.linspace(xspan[0], xspan[1], k+2)
uplot         = np.zeros(k+2)
uplot[1:k+1]  = u

# First plot
fig = plt.figure()
plt.plot(xplot, uplot, 'r')
plt.axis([xspan[0], xspan[1], -2, 2])
plt.xlabel('x')
plt.ylabel('u')
plt.title('time %f' % (tspan[0]))
ax = plt.gca()
points = ax.lines[0]
plot_freq = 10

def one_frame(num):
    global u, uprev, uplot, points
    points.set_ydata(uplot)
    plt.gca().set_title('time %.1f' % (num * plot_freq * dt))
    for i in range(plot_freq):
        unew  = 2*u  -  uprev  -  c * (dt/dx)**2 * A @ u
        uprev = u
        u     = unew
    uplot[1:k+1] = u
    return points,

u = u0

wave_ani = animation.FuncAnimation(fig, one_frame, int(tsteps/plot_freq),
                                   interval=50, blit=True)
wave_ani.save('test.mp4', writer=writer)
        

In [ ]:
# Wave equation in 1D, fully discrete with centered differences in both time and space
# Initial condition is a sum of random Gaussians, different every time it's run

import numpy as np
import scipy
import scipy.sparse.linalg as spla
from scipy import sparse
from scipy import linalg
from scipy import integrate
from scipy.stats import norm

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation

plt.rcParams['animation.ffmpeg_path'] = '/Users/gilbert/anaconda3/bin/ffmpeg'
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

# Wave equation in 1D, fully discrete with centered differences in both time and space
#   d^2u / dt^2 = c * d^2u / dx^2,
#   with boundary values of 0 at the endpoints of unit interval,
#   initial values for u to be specified in the interior of the unit interval,
#   initial values of 0 for du/dt at all points in unit interval, 
#   and (for now) constant propagation speed c.

# Parameters of discretization and the equation
xspan  = (0, 1)         # Left and right endpoints of the 1-D interval
dx     = .001           # Distance between discrete points on the line
k      = int((xspan[1] - xspan[0]) / dx) - 1   # Number of discrete points in space, not including endpoints

tspan  = (0, 4)          # Starting and stopping times
dt     = .0005           # Length of time step 
tsteps = int((tspan[1] - tspan[0]) / dt) + 1   # Number of time steps

c = 1                   # Speed of wave propagation

# Get the matrix for the discrete Laplace operator
A      = make_A_1D(k)

# Initial condition: value of interior points at time 0

u0 = np.zeros(k)
for i in range(10):
    u0 += 50*np.random.normal() * norm.pdf(range(k), 
                                           loc = int(k*np.random.random()), 
                                           scale = k/25 * np.random.random())

# Intialize u and uprev to the same value, 
# because of the initial condition du/dt = 0 everywhere
u     = u0
uprev = u

# For plotting, we include the endpoints in an array xplot of k+2 discrete points,
# and an array uplot of the k interior values along with the 2 boundary values of 0.
xplot         = np.linspace(xspan[0], xspan[1], k+2)
uplot         = np.zeros(k+2)
uplot[1:k+1]  = u

# First plot
fig = plt.figure()
plt.plot(xplot, uplot, 'r')
plt.axis([xspan[0], xspan[1], -2, 2])
plt.xlabel('x')
plt.ylabel('u')
plt.title('time %f' % (tspan[0]))
ax = plt.gca()
points = ax.lines[0]
plot_freq = 10

def one_frame(num):
    global u, uprev, uplot, points
    points.set_ydata(uplot)
    plt.gca().set_title('time %.1f' % (num * plot_freq * dt))
    for i in range(plot_freq):
        unew  = 2*u  -  uprev  -  c * (dt/dx)**2 * A @ u
        uprev = u
        u     = unew
    uplot[1:k+1] = u
    return points,

u = u0

wave_ani = animation.FuncAnimation(fig, one_frame, int(tsteps/plot_freq),
                                   interval=50, blit=True)
wave_ani.save('test.mp4', writer=writer)
        